In [1]:
import pandas as pd
import numpy as np

In [2]:
data_source = "https://archive.ics.uci.edu/ml/machine-learning-databases/cmc/cmc.data"

names = ["Wife's age", "Wife's education", "Husband's education", "Number of children ever born", "Wife's religion", 
"Wife's now working?", "Husband's occupation", "Standard-of-living index", "Media exposure", "Contraceptive method used"]

dataframe = pd.read_csv(data_source, names=names)

In [3]:
dataframe[:5]

,Wife's age,Wife's education,Husband's education,Number of children ever born,Wife's religion,Wife's now working?,Husband's occupation,Standard-of-living index,Media exposure,Contraceptive method used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1
3,42,3,2,9,1,1,3,3,0,1
4,36,3,3,8,1,1,3,2,0,1


In [4]:
dataframe.dtypes

Wife's age                      int64
Wife's education                int64
Husband's education             int64
Number of children ever born    int64
Wife's religion                 int64
Wife's now working?             int64
Husband's occupation            int64
Standard-of-living index        int64
Media exposure                  int64
Contraceptive method used       int64
dtype: object

In [5]:
for column in range(0,10):
    if column not in [0,3]:
        dataframe[names[column]] = dataframe[names[column]].astype("category")

dataframe.dtypes
# y_data = df["Contraceptive method used"].values.reshape(-1,1)

# y_data[:5]

Wife's age                         int64
Wife's education                category
Husband's education             category
Number of children ever born       int64
Wife's religion                 category
Wife's now working?             category
Husband's occupation            category
Standard-of-living index        category
Media exposure                  category
Contraceptive method used       category
dtype: object

In [6]:
y_data = dataframe.pop("Contraceptive method used")

In [7]:
x_data = pd.get_dummies(dataframe).values
y_data = pd.get_dummies(y_data).values

In [8]:
from sklearn import preprocessing as p

min_max_scaler=p.MinMaxScaler()
x_data=min_max_scaler.fit_transform(x_data)
x_data = np.insert(x_data, 0 , 1, axis=1)

In [11]:
idx=np.random.randint(y_data.shape[0],size=int(y_data.shape[0]))
training_idx = idx[:int(y_data.shape[0]*0.8)]
test_idx = idx[int(y_data.shape[0]*0.8):]

x_training, x_test = x_data[training_idx], x_data[test_idx]
y_training, y_test = y_data[training_idx], y_data[test_idx]

(1178, 25)

In [20]:
y_training.shape

(1178, 3)

In [25]:
import tensorflow as tf

x = x_training.T

W = tf.Variable(tf.random_uniform([len(y_training[0]),len(x)], -1.0, 1.0))

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

hypothesis = tf.nn.softmax(tf.matmul(W, X))

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), reduction_indices=1))

W_val = []
cost_val = []

a = tf.Variable(0.01)
optimizer = tf.train.GradientDescentOptimizer(a) # 라이브러리
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

y_training = y_training.T

for step in range(20000000):
    sess.run(train, feed_dict={X:x, Y:y_training})
    if step % 400000 == 0:
        print (step, sess.run(cost, feed_dict={X:x, Y:y_training}), sess.run(W))

0 2822.42 [[ 0.18559086 -0.14107856  0.04733435 -0.05350918 -0.42559323 -0.27720731
  -0.45840025  0.06227974  0.07628897 -0.61867332  0.06104058 -0.0389014
   0.14848554  0.70279074  0.51469028 -0.31459111 -0.24753004  0.04180408
   0.59948927 -0.3205995  -0.41701293 -0.23680966 -0.81275427 -0.80443251
  -0.39883113]
 [-0.58551425 -0.55408323 -0.87555307  0.56212878 -0.14443511 -0.40943176
   0.38472864  0.40646541 -0.70400113  0.03005285  0.36329263 -0.24282426
  -0.87443477  0.41361141 -0.04314591 -0.31940848 -0.33573711 -0.73625195
   0.66514951 -0.81694794  0.11287341 -0.28440073 -0.63176    -0.03463629
   0.69335693]
 [-0.0536845  -0.86998504 -0.39401835 -0.140241    0.2963869  -0.11635865
  -0.29325888  0.52347839 -0.48545787 -0.4942522  -0.42000312 -0.01699939
   0.35131323 -0.3117882   0.33173519  0.3235988  -0.41020206  0.43093783
   0.33298799 -0.34214488 -0.62965339 -0.05886574  0.53711802 -0.74103743
   0.32605964]]


KeyboardInterrupt: 